In [1]:
import argparse
import os
import random
import joblib
from typing import Tuple, Dict, Any
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
! pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.6 MB/s eta 0:00:00
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=401

In [3]:
# InterpretML
from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
# Optuna for Bayesian-style optimization
import optuna

# PyTorch for self-supervised pretraining
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [4]:
import matplotlib.pyplot as plt
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [5]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [6]:
def equalized_odds_difference(y_true, y_pred_proba, sensitive: np.ndarray, threshold=0.5):
    """Max absolute difference in TPR and FPR across groups (binary sensitive)."""
    preds = (y_pred_proba >= threshold).astype(int)
    groups = np.unique(sensitive)
    tprs = []
    fprs = []
    for g in groups:
        mask = (sensitive == g)
        if mask.sum() == 0:
            tprs.append(0.0)
            fprs.append(0.0)
            continue
        tp = ((y_true == 1) & (preds == 1) & mask).sum()
        fn = ((y_true == 1) & (preds == 0) & mask).sum()
        fp = ((y_true == 0) & (preds == 1) & mask).sum()
        tn = ((y_true == 0) & (preds == 0) & mask).sum()
        tprs.append(tp / (tp + fn) if (tp + fn) > 0 else 0.0)
        fprs.append(fp / (fp + tn) if (fp + tn) > 0 else 0.0)
    # Differences
    tpr_diff = abs(tprs[0] - tprs[1]) if len(tprs) >= 2 else 0.0
    fpr_diff = abs(fprs[0] - fprs[1]) if len(fprs) >= 2 else 0.0
    return max(tpr_diff, fpr_diff)

In [7]:
def empirical_robustness_noise(model_predict_proba, X, y, noise_std_list=[0.01, 0.05, 0.1]):
    """Simple robustness: drop in ROC AUC under additive Gaussian noise."""
    base_auc = roc_auc_score(y, model_predict_proba(X))
    results = {'base_auc': base_auc}
    for std in noise_std_list:
        Xp = X + np.random.normal(0, std, size=X.shape)
        auc = roc_auc_score(y, model_predict_proba(Xp))
        results[f'auc_noise_{std}'] = auc
    return results


In [8]:
def load_adult_dataset() -> Tuple[pd.DataFrame, pd.Series, np.ndarray]:
    """Load Adult dataset directly from UCI repository."""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    df = pd.read_csv(
        url,
        header=None,
        names=[
            "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
            "MaritalStatus", "Occupation", "Relationship", "Race", "Sex",
            "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
        ]
    )

    # Label mapping: ">50K" -> 1, "<=50K" -> 0
    df['Income'] = df['Income'].str.strip()
    y = df['Income'].apply(lambda s: 1 if s == '>50K' else 0).astype(int)

    # Features
    X = df.drop(columns=['Income'])

    # Sensitive attribute (as before): "Sex"
    sensitive = (X['Sex'].str.strip() == 'Male').astype(int).values

    return X, y, sensitive

In [9]:
DATASET_LOADERS = {
    'adult': load_adult_dataset,
}


In [10]:
def build_adult_preprocessor(X: pd.DataFrame):
    return FunctionTransformer(lambda x: x)

In [11]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

def format_n(x):
    return "{0:.3f}".format(x)

def train_baseline_ebm_with_vis(X, y, params=None):
    params = params or {}
    model = ExplainableBoostingClassifier(**params)

    ss = StratifiedShuffleSplit(n_splits=3, test_size=0.25, random_state=1337)
    scores = cross_validate(
        model, X, y, scoring='roc_auc', cv=ss,
        n_jobs=None, return_estimator=True, return_train_score=False
    )

    record = dict()
    record['fit_time_mean'] = format_n(np.mean(scores['fit_time']))
    record['fit_time_std'] = format_n(np.std(scores['fit_time']))
    record['test_score_mean'] = format_n(np.mean(scores['test_score']))
    record['test_score_std'] = format_n(np.std(scores['test_score']))

    # Take the first trained model
    trained_model = scores['estimator'][0]

    # Get one split
    train_idx, test_idx = list(ss.split(X, y))[0]

    # Handle pandas vs numpy gracefully
    if hasattr(X, "iloc"):  # Pandas DataFrame
        X_test = X.iloc[test_idx]
    else:  # NumPy array
        X_test = X[test_idx]

    if hasattr(y, "iloc"):  # Pandas Series
        y_test = y.iloc[test_idx]
    else:  # NumPy array
        y_test = y[test_idx]

    # Confusion matrix
    y_pred = trained_model.predict(X_test)
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    plt.title("Confusion Matrix (Validation Split)")
    plt.show()

    # InterpretML visualization
    ebm_global = trained_model.explain_global()
    show(ebm_global)

    return trained_model, record


In [12]:
class TabularAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, max(64, latent_dim * 4)),
            nn.ReLU(),
            nn.Linear(max(64, latent_dim * 4), latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, max(64, latent_dim * 4)),
            nn.ReLU(),
            nn.Linear(max(64, latent_dim * 4), input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return recon, z


In [13]:
def pretrain_autoencoder_get_init_scores(X_unlabeled: np.ndarray, X_small_labeled: np.ndarray, y_small: np.ndarray,
                                         device='cpu', epochs=50, batch_size=256, latent_dim=32):
    """
    Train autoencoder on X_unlabeled (self-supervised), then train a small classifier on
    the encoder output for the labeled subset to produce init_score probabilities.
    Returns init_score for entire dataset (probabilities).
    """
    device = torch.device(device)
    X_unlabeled_t = torch.from_numpy(X_unlabeled.astype(np.float32))
    ds = TensorDataset(X_unlabeled_t)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

    ae = TabularAutoencoder(input_dim=X_unlabeled.shape[1], latent_dim=latent_dim).to(device)
    opt = torch.optim.Adam(ae.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()

    for ep in range(epochs):
        ae.train()
        epoch_loss = 0.0
        for (batch,) in dl:
            batch = batch.to(device)
            recon, _ = ae(batch)
            loss = loss_fn(recon, batch)
            opt.zero_grad()
            loss.backward()
            opt.step()
            epoch_loss += loss.item() * batch.size(0)
    # build encoder outputs for labeled small set
    ae.eval()
    with torch.no_grad():
        Z_small = ae.encoder(torch.from_numpy(X_small_labeled.astype(np.float32)).to(device)).cpu().numpy()
        Z_full = ae.encoder(torch.from_numpy(X_unlabeled.astype(np.float32)).to(device)).cpu().numpy()

    # train logistic regression on Z_small -> produce probabilities for full set as init_score
    clf = LogisticRegression(max_iter=1000)
    clf.fit(Z_small, y_small)
    init_scores = clf.predict_proba(Z_full)[:, 1]
    return init_scores, ae, clf

In [14]:
output_dir='out_M_Preprocessing'
dataset='adult'

In [15]:
def get_adult_data():
    os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Loading {dataset} dataset ...')
    X_df, y, sensitive = DATASET_LOADERS[dataset]()
    preproc = build_adult_preprocessor(X_df)
    X_all = preproc.fit_transform(X_df)
    # convert to numpy arrays for later use
    #X_all = np.asarray(X_all, dtype=float)
    y = np.asarray(y, dtype=int)
    print("[INFO] Loading completed")

   

    return X_all,y

In [16]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def run_hpo_ebm_without_dp(X_train, y_train, n_trials=40, timeout=None, output_dir='hpo'):
    os.makedirs(output_dir, exist_ok=True)

    def objective(trial):
        # sample hyperparams
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
            'max_bins': trial.suggest_int('max_bins', 64, 512),
            'max_leaves': trial.suggest_int('max_leaves', 2, 64),
            'max_rounds': trial.suggest_int('max_rounds', 50, 2000),
            'interactions': trial.suggest_int('interactions', 0, 10),
            'outer_bags': trial.suggest_int('outer_bags', 4, 32),
            'inner_bags': trial.suggest_int('inner_bags', 0, 8),
            # greedy ratio can be tuned too
            'greedy_ratio': trial.suggest_float('greedy_ratio', 0.0, 20.0),
        }
      

        model = ExplainableBoostingClassifier(**params)
        #model.fit(X_train, y_train)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        roc = cross_val_score(model, X_train, y_train, cv=cv, scoring="roc_auc").mean()
      
        objective_value = (1.0 - roc) 

        # store additional info as trial user attributes
        trial.set_user_attr('roc_auc', roc)
       
        trial.set_user_attr('params', params)
        return objective_value

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(objective, n_trials=n_trials, timeout=timeout)
    # persist study
    joblib.dump(study, os.path.join(output_dir, 'optuna_study_without_dp.pkl'))
    return study


In [17]:
X_all, y = get_adult_data()
print('[INFO] Running HPO (Optuna)...')
study = run_hpo_ebm_without_dp(X_all, y, n_trials=30, output_dir=output_dir)
print('[HPO] Best trial:', study.best_trial.params)

[INFO] Loading adult dataset ...


[I 2025-09-01 18:39:54,434] A new study created in memory with name: no-name-e22d9095-daa2-4847-bb11-e8bafde18273


[INFO] Loading completed
[INFO] Running HPO (Optuna)...


/tmp/ipykernel_13/3030925094.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2025-09-01 18:41:39,291] Trial 0 finished with value: 0.07272485857649946 and parameters: {'learning_rate': 0.0013292918943162175, 'max_bins': 490, 'max_leaves': 48, 'max_rounds': 1217, 'interactions': 1, 'outer_bags': 8, 'inner_bags': 0, 'greedy_ratio': 17.323522915498703}. Best is trial 0 with value: 0.07272485857649946.
/tmp/ipykernel_13/3030925094.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2025-09-01 18:44:39,398] Trial 1 finished

[HPO] Best trial: {'learning_rate': 0.053837171670457214, 'max_bins': 239, 'max_leaves': 8, 'max_rounds': 908, 'interactions': 7, 'outer_bags': 20, 'inner_bags': 0, 'greedy_ratio': 10.890167671001365}
